In [1]:
import os
os.chdir("c:\\Users\\Eduardo Pacheco\\Desktop\\Arquivos\\Personal\\SkateAI")

In [2]:
import torch
import numpy as np
import pandas as pd
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader

from models import ResNet18_RNN
from dataset import TricksDataset, VideoToTensor

In [3]:
resnet_transforms = models.resnet.ResNet18_Weights.DEFAULT.transforms()

In [4]:
LABELS = [
    "body_rotation_type",
    "body_rotation_number",
    "board_rotation_type",
    "board_rotation_number",
    "flip_type",
    "flip_number",
    "landed",
    "stance"
]

In [5]:
df = pd.read_csv("data/metadata/metadata.csv")
df.head()

,video_file,video_title,video_url,video_source,trick_interval,trick_name,body_rotation_type,body_rotation_number,board_rotation_type,board_rotation_number,flip_type,flip_number,landed,stance
0,00001.mp4,BATB 11 | Championship Battle: Luan Oliveira v...,https://www.youtube.com/watch?v=sDLIf7JVNnM,BATB 11,"[149.6, 151.9]",treflip,0,0,1,2,1,1,1,0
1,00002.mp4,BATB 11 | Championship Battle: Luan Oliveira v...,https://www.youtube.com/watch?v=sDLIf7JVNnM,BATB 11,"[154.2, 155.1]",treflip,0,0,1,2,1,1,1,0
2,00003.mp4,BATB 11 | Championship Battle: Luan Oliveira v...,https://www.youtube.com/watch?v=sDLIf7JVNnM,BATB 11,"[159.4, 161.2]",kickflip,0,0,0,0,1,1,1,3
3,00004.mp4,BATB 11 | Championship Battle: Luan Oliveira v...,https://www.youtube.com/watch?v=sDLIf7JVNnM,BATB 11,"[163.8, 166.0]",kickflip,0,0,0,0,1,1,1,3
4,00005.mp4,BATB 11 | Championship Battle: Luan Oliveira v...,https://www.youtube.com/watch?v=sDLIf7JVNnM,BATB 11,"[169.5, 171.1]",kickflip,0,0,0,0,1,1,1,0


In [6]:
transformations = transforms.Compose(
    [
        resnet_transforms
    ]
)
tricks_dataset = TricksDataset(csv_file="data\\metadata\\metadata.csv", root_dir="data\\videos", max_frames=69, transform=transformations)

In [7]:
for video, _ in tricks_dataset:
    break

In [8]:
loader = DataLoader(tricks_dataset, batch_size=4)

In [9]:
videos_batch, targets_batch = next(iter(loader))

In [10]:
targets_batch

{'body_rotation_type': tensor([0, 0, 0, 0]),
 'body_rotation_number': tensor([0, 0, 0, 0]),
 'board_rotation_type': tensor([1, 1, 0, 0]),
 'board_rotation_number': tensor([2, 2, 0, 0]),
 'flip_type': tensor([1, 1, 1, 1]),
 'flip_number': tensor([1, 1, 1, 1]),
 'landed': tensor([1, 1, 1, 1]),
 'stance': tensor([0, 0, 3, 3])}

In [11]:
MAX_FRAMES = 69
RNN_TYPE = "lstm"
RNN_LAYERS = 2
RNN_HIDDEN = 100
HEADS_PARAMS = {
    "in_features": RNN_HIDDEN*MAX_FRAMES,
    "byrt": [2, 2],
    "byrn": [2, 2],
    "bdrt": [2, 2],
    "bdrn": [2, 2],
    "ft": [2, 2],
    "fn": [2, 2],
    "landed": [2, 2],
    "stance":[2, 2]
}
TRAINABLE_BACKBONE = False

In [12]:
1 + torch.Tensor([10])

tensor([11.])

In [13]:
targets_batch.keys()

dict_keys(['body_rotation_type', 'body_rotation_number', 'board_rotation_type', 'board_rotation_number', 'flip_type', 'flip_number', 'landed', 'stance'])

In [14]:
model = ResNet18_RNN(
    RNN_TYPE, 
    RNN_LAYERS, 
    RNN_HIDDEN, 
    HEADS_PARAMS, 
    TRAINABLE_BACKBONE
)

In [15]:
outs = model(videos_batch)